In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv('GROQ_API_KEY1')

In [2]:
llm = ChatGroq(model='Gemma2-9b-It' ,
         groq_api_key=GROQ_API_KEY)

In [3]:
from langchain_core.messages import SystemMessage , AIMessage , HumanMessage
llm.invoke([
    HumanMessage(content="Hi , My name is Abishek and I am a Student")
])

AIMessage(content="Hi Abishek, \n\nIt's nice to meet you!\n\nWhat can I do for you today? Are you working on a school project, looking for information, or just want to chat? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 22, 'total_tokens': 70, 'completion_time': 0.087272727, 'prompt_time': 0.001323221, 'queue_time': 0.25149079, 'total_time': 0.088595948}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7a90c3ad-7559-4538-9adc-648eb4291bd1-0', usage_metadata={'input_tokens': 22, 'output_tokens': 48, 'total_tokens': 70})

In [4]:
llm.invoke([
    HumanMessage(content="Hi , My name is Abishek and I am a Student"),
    AIMessage(content="Hi Abishek, \n\nIt's nice to meet you!\n\nWhat can I do for you today? Are you working on a school project, looking for information, or just want to chat? 😊"),
    HumanMessage(content="What is my Name ? and do i have a job?")
])

AIMessage(content="You told me your name is Abishek!  \n\nAnd based on what you said, you are a student.  That means you don't have a job, right? \n\nIs there anything else you'd like to talk about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 87, 'total_tokens': 141, 'completion_time': 0.098181818, 'prompt_time': 0.002500561, 'queue_time': 0.25605027399999997, 'total_time': 0.100682379}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b4a743c9-dc8b-4d8a-b18b-bf5ef239ee46-0', usage_metadata={'input_tokens': 87, 'output_tokens': 54, 'total_tokens': 141})

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store={}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(llm , get_session_history)

In [13]:
configuration = {"configurable":{"session_id": "chat1"}}

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Abishek and I'm a student")],
    config=configuration
)

In [16]:
response.content

"Hi Abishek!  Nice to meet you.\n\nWhat can I do for you today? Are you working on a project, or just curious about something?  I'm happy to help in any way I can. 😊\n"

In [18]:
with_message_history.invoke(
    [HumanMessage(content="What is my Name?")],
    config=configuration
)

AIMessage(content='Your name is Abishek.  \n\nI remember that from our first conversation!  😄  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 86, 'total_tokens': 119, 'completion_time': 0.06, 'prompt_time': 0.00259234, 'queue_time': 0.25719112, 'total_time': 0.06259234}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--497e69f2-ca31-47c5-9020-2b0e9b1801e2-0', usage_metadata={'input_tokens': 86, 'output_tokens': 33, 'total_tokens': 119})

In [20]:
configuration1 = {"configurable":{"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi, What is my name")],
    config=configuration1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know! 😊\n"

## Prompt Template

In [21]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , "You are a Helpful Assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name='messages')
    ]
)
chain = prompt | llm

In [23]:
chain.invoke({"messages": [HumanMessage(content="How to feel someone special ?")]})

AIMessage(content="It's wonderful that you want to make someone feel special! It's about showing them you care and appreciate them. Here are some ways to do that:\n\n**Thoughtful Gestures:**\n\n* **Listen attentively:**  When they talk, really listen.  Make eye contact, ask questions, and show genuine interest.\n* **Remember the little things:**  Recall their favorite things, like their birthday, hobbies, or a funny inside joke. \n* **Small acts of kindness:** Offer to help with a task, make them their favorite meal, or surprise them with a small gift that shows you were thinking of them.\n* **Write a heartfelt note:**  A handwritten letter or card expressing your appreciation can be incredibly meaningful.\n\n**Quality Time:**\n\n* **Spend time doing something they enjoy:**  Even if it's not your favorite thing, showing interest in their hobbies shows you care.\n* **Be present:** When you're together, put away your phone and focus on them. \n* **Create new memories:**  Try a new activi

In [24]:
with_message_history = RunnableWithMessageHistory(chain , get_session_history)

In [25]:
configuration1 = {"configurable":{"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Abishek")],
    config=configuration1
)
response.content

"Hello Abishek, it's nice to meet you! \n\nWhat can I help you with today? 😊  \n\n"

In [26]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , "You are a Helpful Assistant. Answer all the question to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name='messages')
    ]
)
chain = prompt | llm

In [27]:
response = chain.invoke({"messages":[HumanMessage(content="HI, My name is Abishek  and i really like someone how to express..?")],"language":"Hindi"})

In [28]:
response

AIMessage(content='नमस्ते अभिषेक! 😊 \n\nकिसी को पसंद करना बहुत ही अच्छा महसूस होता है!  \n\nआप अपनी भावनाएँ व्यक्त करने के लिए कई तरह के तरीके अपना सकते हैं। \n\n**कुछ सुझाव:**\n\n* **सबसे पहले, अपने आप से यकीन कर लें:**  क्या आपको वास्तव में उन्हें पसंद हैं, या यह सिर्फ एक चंचल मनोदशा है? \n\n* **धीरे-धीरे शुरुआत करें:**  उन्हें एक मीठी बात कहें,  एक छोटा सा उपहार दें, या उनके साथ ज्यादा समय बिताने की कोशिश करें। \n\n* **अपनी दिलचस्पी दिखाएं:**  उनके बारे में पूछें, उनकी बातों में दिलचस्पी लें, और उनके शौक और रुचियों को समझने की कोशिश करें।\n\n* **उनके साथ एक-एक करके जुड़ें:**  उनके साथ सामान्य बातचीत करें,  उनकी हँसी सुनें, और उनसे गहरी बातचीत करने का प्रयास करें। \n\n* **विश्वास बनाएँ:**  उनसे ईमानदारी से बात करें, और अपने विचारों और भावनाओं को साझा करें। \n\n* **स्पष्ट रहें:**  अगर आप उन्हें पसंद करते हैं, तो उन्हें बताएं!  \n\nयाद रखें, धैर्य रखें और सम्मानजनक बनें। \n\nउम्मीद है कि ये सुझाव आपको मददगार होंगे! \n\n', additional_kwargs={}, response_metadata={'token_usage': {'comple

In [29]:
with_message_history = RunnableWithMessageHistory(chain,
                                                get_session_history,
                                                input_messages_key="messages")

In [30]:
configuration4 = {"configurable":{"session_id": "chat4"}}
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi, My name is Abishek")],"language":"Hindi"},
    config=configuration4
)
response.content


'Namaste Abishek! \n\nआप मुझसे जो भी प्रश्न पूछेंगे, मैं उसका उत्तर देने की पूरी कोशिश करूँगा। 😊  \n\nआप किस बारे में जानना चाहते हैं? \n'

In [31]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="What is my name?")],"language":"Hindi"},
    config=configuration4
)
response.content

'आपका नाम Abishek है। \n'

### Manage conversation History

In [43]:
def my_token_counter(text: str)-> int:
    return len(text.split())

In [46]:
from langchain_core.messages import trim_messages
trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on ="human"
)

In [39]:
messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Hi, my name is Abishek."),
    AIMessage(content="Hello Abishek! How can I help you today?"),
    HumanMessage(content="Can you tell me about artificial intelligence?"),
    AIMessage(content="Sure! Artificial intelligence is a field of computer science focused on creating systems that can perform tasks that typically require human intelligence."),
    HumanMessage(content="That's interesting. Can you give an example?"),
    AIMessage(content="Of course! Virtual assistants like Siri or Alexa are examples of AI that help users with various tasks."),
    HumanMessage(content="Thank you!"),
    AIMessage(content="You're welcome! If you have more questions, feel free to ask.")
]

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | llm
)
response=chain.invoke(
    {
        "messages": messages+ [HumanMessage(content="What topic did i ask about?")],
        "language": "English",

    }
)
response.content